# Social Distance Detection using scipy.spatial and OpenCV

#  Importing libraries and ssd_mobilenet_v3 configs

In [10]:
import cv2 
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist

In [11]:
model='frozen_inference_graph.pb'
config='ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt.txt'
labels=[]
models=cv2.dnn_DetectionModel(model,config)

In [12]:
file='coco.txt'
with open(file,'r') as fp:
    labels=fp.read().rstrip('\n').split('\n')
    if labels=='person':
        label='person'

In [13]:
print(labels)
len(labels)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


80

# initialising the model's configuration

In [14]:
models.setInputScale(1.0/127.5)
models.setInputSize(320,320)
models.setInputMean((127.5,127.5,127.5))
models.setInputSwapRB(True)

<dnn_Model 0000020D017ED2F0>

# in this section we are observing the video and storing the information regarding people's coordinates 

In [15]:
violate=set()
listy=[]
cam =cv2.VideoCapture("CCTV_demo.mp4")
if not cam.isOpened():
    cam =cv2.VideoCapture("CCTV_demo.mp4")
    if not cam.isOpened():
        print('error in camera')
while(True):
    ret,frame=cam.read()
    Classind ,confidence, blob = models.detect(frame,confThreshold=0.6)
    #print( Classind )
    if (len(Classind)!=0):
        for Class,conf,bl in zip(Classind.flatten(),confidence.flatten(),blob):
            if Class==1:
                listy.append(bl)
                cv2.rectangle(frame,bl,(255,0,0),3)
                cv2.putText(frame,labels[Class-1],(bl[0]+10,bl[1]+40),4,1,250,4)
    cv2.imshow('frame',frame)
    if cv2.waitKey(2) & 0xFF ==ord('q'):

        break

        
cam.release()
cv2.destroyAllWindows()
        
        

# in this section we are calculating the distance through euclidean distance 

In [16]:
centroid=[]
results=[]
MIN_DISTANCE= 50
for i in range(len(listy)):
    x=listy[i][0]  
    y=listy[i][1]
    w=listy[i][2]
    h=listy[i][3]
    centroid.append([x,y])
    r= ((x, y, x + w, y + h), centroid[i])
    results.append(r)
                #v2.rectangle(frame,bl,(255,0,0),3)
               #cv2.putText(frame,labels[Class-1],(bl[0]+10,bl[1]+40),1,1,250,4)
                
    #2.imshow('frame',frame)
D = dist.cdist(centroid, centroid, metric="euclidean")

     #oop over the upper triangular of the distance matrix
for i in range(0, D.shape[0]):
    for j in range(i + 1, D.shape[1]):
        if D[i, j] < MIN_DISTANCE:
            # update our violation set with the indexes of
            # the centroid pairs
            violate.add(D[i,j])
             
        

# THE FINAL OUTPUT AND SOCIAL DISTANCING VOILATION COUNTER

In [18]:
output1 = "Output_file.avi"
cam =cv2.VideoCapture("CCTV_demo.mp4")
if not cam.isOpened():
    cam =cv2.VideoCapture("CCTV_demo.mp4")
    if not cam.isOpened():
        print('error in camera')
while(True):
    ret,frame=cam.read()
    Classind ,confidence, blob = models.detect(frame,confThreshold=0.6)
    result=zip(Classind.flatten(),confidence.flatten(),blob)
    for (i,(Class,conf,bl)) in enumerate(result):
        if Class==1:
            color = (0, 255, 0)
            if i in violate:
                color = (0, 0, 255)
            cv2.rectangle(frame,bl,color,3)
            #cv2.circle(frame, centroid[i], 5, color, 1)
            cv2.putText(frame,labels[Class-1],(bl[0]+10,bl[1]+40),4,1,250,4)
               
                
    text = "Social Distancing Violations: {}".format(i)
    cv2.putText(frame, text, (10, frame.shape[0] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

        # if the index pair exists within the violation set, then
        # update the color
           
    cv2.imshow('frame',frame)
    if cv2.waitKey(2) & 0xFF ==ord('q'):
        break
    
    fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
    out = cv2.VideoWriter("output2.avi", fourcc, 5.0, (1280,720))
        
cam.release()
cv2.destroyAllWindows()
        
        